In [5]:
import pandas as pd
import ast
import os as os

In [8]:
current_directory = os.getcwd()
current_directory

'C:\\Users\\andre\\PycharmProjects\\dmmlProject\\progetto'

In [9]:
# Function to process the cast entries
def process_cast(cast_data, film_id):
    cast_entries = []
    for cast_member in cast_data:
        department = 'voice over' if '(voice)' in cast_member['character'] else 'actors'
        job = 'voice actor' if '(voice)' in cast_member['character'] else 'actor'
        cast_entries.append({
            'film_id': film_id,
            'cast_id': cast_member['cast_id'],
            'id': cast_member['id'],
            'credit_id': cast_member['credit_id'],
            'actor_name': cast_member['name'],
            'actor_gender': cast_member['gender'],
            'character_name': cast_member['character'].replace('(voice)', '').strip(),
            'order': cast_member['order'],
            'profile_path': cast_member['profile_path'],
            'department': department,
            'job': job
        })
    return cast_entries

# Function to process the crew entries
def process_crew(crew_data, film_id):
    crew_entries = []
    for crew_member in crew_data:
        crew_entries.append({
            'film_id': film_id,
            'cast_id': -1,
            'id': crew_member['id'],
            'credit_id': crew_member['credit_id'],
            'actor_name': crew_member['name'],
            'actor_gender': crew_member['gender'],
            'character_name': 'enoN',
            'order': -1,
            'profile_path': crew_member['profile_path'],
            'department': crew_member['department'],
            'job': crew_member['job']
        })
    return crew_entries

# Read the original CSV file
file_path = 'dataset/credits.csv'
credits_df = pd.read_csv(file_path)

# Create a new DataFrame for the processed data
new_credits_data = []

# Process each entry in the original DataFrame
for index, row in credits_df.iterrows():
    film_id = row['id']
    cast_data = ast.literal_eval(row['cast'])
    crew_data = ast.literal_eval(row['crew'])
    
    # Process cast and crew, extending the new data list with the results
    new_credits_data.extend(process_cast(cast_data, film_id))
    new_credits_data.extend(process_crew(crew_data, film_id))

# Convert the new data list to a DataFrame
new_credits_df = pd.DataFrame(new_credits_data)

# Save the new DataFrame to a CSV file
new_credits_file_path = 'dataset/newCredits.csv'
new_credits_df.to_csv(new_credits_file_path, index=False)

new_credits_file_path


'dataset/newCredits.csv'

In [10]:
keywords_df = pd.read_csv(os.path.join('dataset','keywords.csv'))
keywords_df

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [13]:
keywords_df = pd.read_csv('dataset/keywords.csv')

# This function will parse the 'keywords' column and return a list of dictionaries
# with the film_id, keyword_id, and keyword for each keyword entry.
def parse_keywords(row):
    try:
        # This assumes that the 'keywords' column is a string representation of a list of dictionaries.
        keywords_list = ast.literal_eval(row['keywords'])
        keywords_list
        return [
            {'film_id': row['id'], 'keyword_id': keyword['id'], 'keyword': keyword['name']}
            for keyword in keywords_list
        ]
    except Exception as e:
        # Handle exceptions that may occur during parsing
        print(f"Error parsing row {row['id']}: {e}")
        return []

# Use the function to expand the 'keywords' column into a list of dictionaries
new_keywords_data = []
for index, row in keywords_df.iterrows():
    new_keywords_data.extend(parse_keywords(row))

# Convert the list of dictionaries into a new DataFrame
new_keywords_df = pd.DataFrame(new_keywords_data)

# Save the new DataFrame to a CSV file
new_keywords_df.to_csv('dataset/newKeywords.csv', index=False)


In [19]:
import pandas as pd
import ast
from collections import Counter

# Replace this file path with the path to your actual movies_metadata.csv file
file_path = 'dataset/movies_metadata.csv'
movies_metadata_df = pd.read_csv(file_path)

genre_counts = Counter()
movies_with_unique_genre = []
movies_with_no_genres = []
movies_with_invalid_genres = []
unique_genre_count = 0

# Function to safely parse genres data
def safe_parse_genres(genres_str):
    try:
        return ast.literal_eval(genres_str) if pd.notnull(genres_str) else []
    except (SyntaxError, ValueError):
        return None

# Iterate through the DataFrame and count genres
for index, row in movies_metadata_df.iterrows():
    genres = safe_parse_genres(row['genres'])

    # If genres is None, then the genres data was invalid
    if genres is None:
        movies_with_invalid_genres.append(row)
        continue

    # If genres list is empty, then the movie has no genres
    if not genres:
        movies_with_no_genres.append(row)
        continue

    # Update genre counts
    for genre in genres:
        genre_counts[genre['name']] += 1

# Find unique genres and their corresponding movies
unique_genres = {genre for genre, count in genre_counts.items() if count == 1}
for index, row in movies_metadata_df.iterrows():
    genres = safe_parse_genres(row['genres'])
    if genres:
        movie_genres = {genre['name'] for genre in genres}
        if movie_genres & unique_genres:
            movies_with_unique_genre.append(row)
            unique_genre_count += 1

# Print movies with a unique genre
print("Movies with a unique genre:")
print(len(movies_with_unique_genre))

# Print movies with no genres or invalid genres
print("\nMovies with no genres:")
print(len(movies_with_no_genres))

print("\nMovies with invalid genres:")
print(len(movies_with_invalid_genres))

# Print the total number of incorrect entries
total_invalid_entries = len(movies_with_no_genres) + len(movies_with_invalid_genres)
print(f"\nEntries non corrette: {total_invalid_entries}")


C:\Users\andre\AppData\Local\Temp\ipykernel_12708\1149187057.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv(file_path)


Movies with a unique genre:
3

Movies with no genres:
2442

Movies with invalid genres:
0

Entries non corrette: 2442


In [24]:
import pandas as pd
import ast

# Load the dataset
file_path = 'dataset/movies_metadata.csv'  # Make sure to provide the correct path to your CSV file
movies_metadata_df = pd.read_csv(file_path)

# Define a function to check if genres are valid (properly formatted JSON)
def is_valid_genre(genres_str):
    try:
        # This will return False if genres_str is NaN or genres is an empty list
        if pd.isna(genres_str) or not genres_str.strip():
            return False
        # Attempt to parse the string into JSON
        genres = ast.literal_eval(genres_str)
        return isinstance(genres, list)
    except:
        # Any error in parsing will mean the genre is invalid
        return False

# Filter out rows where the 'genres' column is not valid
valid_movies_df = movies_metadata_df[movies_metadata_df['genres'].apply(is_valid_genre)]

# Save the new DataFrame to a new CSV file
new_file_path = 'newMovieMetadata.csv'
valid_movies_df.to_csv(new_file_path, index=False)

# Let the user know the new file has been created
print(f"Created new file without invalid genres: {new_file_path}")


C:\Users\andre\AppData\Local\Temp\ipykernel_12708\4233765080.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv(file_path)


Created new file without invalid genres: newMovieMetadata.csv


In [27]:
import pandas as pd

# Load the dataset
file_path = 'newMovieMetadata.csv'  # Replace with the actual file path
movies_metadata_df = pd.read_csv(file_path)

# Function to check if a value is a non-negative number
def is_non_negative_number(value):
    try:
        number = float(value)
        return number > 0
    except (ValueError, TypeError):
        return False

# Count films with invalid budget
invalid_budget_count = movies_metadata_df['budget'].apply(lambda x: not is_non_negative_number(x) or x == 0).sum()

# Count films with invalid revenue
invalid_revenue_count = movies_metadata_df['revenue'].apply(lambda x: not is_non_negative_number(x)).sum()

# Count films with invalid vote_average or vote_count
invalid_votes_count = movies_metadata_df.apply(lambda row: not (is_non_negative_number(row['vote_average']) and is_non_negative_number(row['vote_count'])), axis=1).sum()

# Print the results
print(f"Number of films with invalid budget: {invalid_budget_count}")
print(f"Number of films with invalid revenue: {invalid_revenue_count}")
print(f"Number of films with invalid vote_average or vote_count: {invalid_votes_count}")


C:\Users\andre\AppData\Local\Temp\ipykernel_12708\2122785835.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv(file_path)


Number of films with invalid budget: 36576
Number of films with invalid revenue: 38058
Number of films with invalid vote_average or vote_count: 3004


In [30]:
import pandas as pd

# Load the dataset
file_path = 'newMovieMetadata.csv'  # Replace with the actual file path
movies_metadata_df = pd.read_csv(file_path)

# Define a function to check if a value is a positive number and not NaN, None, or empty
def is_valid(value):
    try:
        return pd.notna(value) and float(value) > 0
    except (ValueError, TypeError):
        return False

# Apply the filter conditions for revenue, budget, vote_average, and vote_count
filtered_movies_df = movies_metadata_df[
    (movies_metadata_df['revenue'].apply(is_valid)) &
    (movies_metadata_df['budget'].apply(is_valid)) 
]

# Save the filtered DataFrame to a new CSV file
new_file_path = 'dataset/tmpMovies.csv'
filtered_movies_df.to_csv(new_file_path, index=False)

print(f"Filtered data saved to {new_file_path}")


C:\Users\andre\AppData\Local\Temp\ipykernel_12708\3789008355.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv(file_path)


Filtered data saved to dataset/tmpMovies.csv


In [31]:
import pandas as pd

# Load the datasets
new_keywords_df = pd.read_csv('dataset/newKeywords.csv')
tmp_movies_df = pd.read_csv('dataset/tmpMovies.csv')

# Aggregate keywords by film_id into a list of dictionaries
keywords_grouped = new_keywords_df.groupby('film_id').apply(lambda x: x[['keyword_id', 'keyword']].to_dict('records')).reset_index(name='keywords')

# Merge the keywords into the tmpMovies dataset based on film_id
tmp_movies_keywords_df = tmp_movies_df.merge(keywords_grouped, left_on='id', right_on='film_id', how='left')

# Drop the extra 'film_id' column as it's redundant
tmp_movies_keywords_df.drop('film_id', axis=1, inplace=True)

# Save the merged dataframe to a new CSV file
output_file_path = 'dataset/tmpMoviesKeywords.csv'
tmp_movies_keywords_df.to_csv(output_file_path, index=False)

output_file_path


C:\Users\andre\AppData\Local\Temp\ipykernel_12708\2068575811.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  keywords_grouped = new_keywords_df.groupby('film_id').apply(lambda x: x[['keyword_id', 'keyword']].to_dict('records')).reset_index(name='keywords')


'dataset/tmpMoviesKeywords.csv'

In [32]:
import pandas as pd

# Carica i dataset tmpMoviesKeywords e credits
tmp_movies_keywords = pd.read_csv('dataset/tmpMoviesKeywords.csv')
credits = pd.read_csv('dataset/credits.csv')

# Unisci i due dataframe basandoti sull'ID del film
tmp_movies_keywords_credits = pd.merge(tmp_movies_keywords, credits, on='id', how='left')

# Seleziona e rinomina le colonne di interesse
tmp_movies_keywords_credits = tmp_movies_keywords_credits[['id', 'cast', 'crew'] + [col for col in tmp_movies_keywords.columns if col != 'id']]

# Salva il nuovo dataframe in un file CSV
tmp_movies_keywords_credits.to_csv('dataset/tmpMoviesKeywordsCredits.csv', index=False)
tmp_movies_keywords_credits.head()


,id,cast,crew,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,tt0114709,en,...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'keyword_id': 931, 'keyword': 'jealousy'}, {..."
1,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,tt0113497,en,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'keyword_id': 10090, 'keyword': 'board game'..."
2,31357,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0114885,en,...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'keyword_id': 818, 'keyword': 'based on nove..."
3,949,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,tt0113277,en,...,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[{'keyword_id': 642, 'keyword': 'robbery'}, {'..."
4,9091,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,tt0114576,en,...,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0,"[{'keyword_id': 949, 'keyword': 'terrorist'}, ..."


In [37]:
import pandas as pd

# Carica il dataset tmpMoviesKeywordsCredits
full_dataset = pd.read_csv('dataset/tmpMoviesKeywordsCredits.csv')

# Rinomina la colonna 'id' in 'film_id'
full_dataset.rename(columns={'id': 'film_id'}, inplace=True)

# Definisci le colonne che vuoi mantenere e il loro ordine
desired_columns = [
    'film_id', 'original_title', 'release_date', 'tagline', 'overview',
    'keywords', 'genres', 'belongs_to_collection', 'original_language', 'spoken_languages',
    'cast', 'crew', 'production_companies', 'production_countries', 'budget',
    'revenue', 'runtime', 'popularity', 'status', 'imdb_id', 'vote_average', 'vote_count'
]

# Filtra il dataframe per mantenere solo le colonne desiderate
full_dataset = full_dataset[desired_columns]

# Salva il nuovo dataframe in un file CSV
full_dataset.to_csv('dataset/full_dataset.csv', index=False)

# Mostra le prime righe del dataframe per confermare la correttezza della formattazione
full_dataset.head()


,film_id,original_title,release_date,tagline,overview,keywords,genres,belongs_to_collection,original_language,spoken_languages,...,production_companies,production_countries,budget,revenue,runtime,popularity,status,imdb_id,vote_average,vote_count
0,862,Toy Story,1995-10-30,NaN,"Led by Woody, Andy's toys live happily in his ...","[{'keyword_id': 931, 'keyword': 'jealousy'}, {...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","{'id': 10194, 'name': 'Toy Story Collection', ...",en,"[{'iso_639_1': 'en', 'name': 'English'}]",...,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",30000000,373554033.0,81.0,21.946943,Released,tt0114709,7.7,5415.0
1,8844,Jumanji,1995-12-15,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"[{'keyword_id': 10090, 'keyword': 'board game'...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",65000000,262797249.0,104.0,17.015539,Released,tt0113497,6.9,2413.0
2,31357,Waiting to Exhale,1995-12-22,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...","[{'keyword_id': 818, 'keyword': 'based on nove...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,en,"[{'iso_639_1': 'en', 'name': 'English'}]",...,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",16000000,81452156.0,127.0,3.859495,Released,tt0114885,6.1,34.0
3,949,Heat,1995-12-15,A Los Angeles Crime Saga,"Obsessive master thief, Neil McCauley leads a ...","[{'keyword_id': 642, 'keyword': 'robbery'}, {'...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",60000000,187436818.0,170.0,17.924927,Released,tt0113277,7.7,1886.0
4,9091,Sudden Death,1995-12-22,Terror goes into overtime.,International action superstar Jean Claude Van...,"[{'keyword_id': 949, 'keyword': 'terrorist'}, ...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,en,"[{'iso_639_1': 'en', 'name': 'English'}]",...,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",35000000,64350171.0,106.0,5.231580,Released,tt0114576,5.5,174.0
